### Bibliotecas

In [73]:
# Bibliotecas
import meteomatics.api as api
import datetime as dt
import os
import numpy as np
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Polygon
import pandas as pd
import cx_Oracle
from funcoes_conexao import conexao_oracle, finaliza_conexao, consulta_para_dataframe, truncate_tabelas
from dotenv import load_dotenv
load_dotenv()

True

### Diretórios

In [74]:
BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(os.path.dirname(BASE_DIR))
DIR_COLETA_DADOS = os.path.join(DIR_PAI, "Coleta de Dados")

In [75]:
# Carregar dados login Oracle
username_ocr = os.getenv('USERNAME_ORC')
password_ocr = os.getenv("PASSWORD_ORC")
dsn = 'ORACLE.FIAP.COM.BR:1521/ORCL'

In [76]:
# Carregar dados login Weather
username = os.getenv('USERNAME_WEATHER_API')
password = os.getenv('PASSWORD_WEATHER_API')

#### Inicializando Conexão ORC

In [77]:
connection, cursor = conexao_oracle(username_ocr, password_ocr, dsn)

#if connection and cursor:
#    finaliza_conexao(cursor, connection)

(3550308, 'São Paulo', 'SP', 1521.11, <cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x3361d6d0>, <cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x36501030>)


#### Limpa Tabelas

In [78]:
#truncate_tabelas(connection, 'TBL_Previsao_Futura')

#### INSERT COORDENADAS - TBL_Previsao_Futura

##### Cosulta região banco de dados

In [79]:
def read_zona_data(connection):
    cursor = connection.cursor()
    
    try:
        # Consultar dados da tabela TBL_ZONA
        cursor.execute("""
            SELECT 
                ID, 
                NOME_ZONA, 
                MUNICIPIO_ID,
                SDO_UTIL.TO_WKTGEOMETRY(GEOMETRY) as geometry_wkt, 
                SDO_UTIL.TO_WKTGEOMETRY(CORD_CENTRAL) as cord_central_wkt 
            FROM TBL_ZONA
        """)
        
        rows = cursor.fetchall()

        # Extrair as colunas dos resultados
        col_names = [desc[0] for desc in cursor.description]

        # Criar um DataFrame com os resultados
        df = pd.DataFrame(rows, columns=col_names)

        # Converter colunas LOB para string e remover as colunas extras
        df['geometry'] = df['GEOMETRY_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else loads(x))
        df['CORD_CENTRAL'] = df['CORD_CENTRAL_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else (loads(x) if x else None))

        # Remover colunas WKT que não são mais necessárias
        df.drop(columns=['GEOMETRY_WKT', 'CORD_CENTRAL_WKT'], inplace=True)

        # Criar um GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')

        return gdf

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao ler dados: {error.message}")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [80]:
# Ler os dados da tabela
Select_TBL_Zona = read_zona_data(connection)
Select_TBL_Zona

Cursor fechado.


,ID,NOME_ZONA,MUNICIPIO_ID,geometry,CORD_CENTRAL
0,1,Centro Ampliado,3550308,"POLYGON ((-46.67421 -23.64222, -46.67437 -23.6...",POINT (-46.679579 -23.582388)
1,2,Zona Leste 1,3550308,"POLYGON ((-46.57536 -23.6008, -46.57552 -23.60...",POINT (-46.542663 -23.561831)
2,3,Zona Leste 2,3550308,"POLYGON ((-46.45643 -23.64132, -46.4565 -23.64...",POINT (-46.443353 -23.549433)
3,4,Zona Norte,3550308,"POLYGON ((-46.67805 -23.51367, -46.67863 -23.5...",POINT (-46.673093 -23.45335)
4,5,Zona Sul,3550308,"POLYGON ((-46.60925 -23.90462, -46.60876 -23.9...",POINT (-46.708599 -23.813758)


In [81]:
Select_TBL_Zona.to_csv('Select_TBL_Zona.csv', index=False)

In [82]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_Previsao_Futura"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,ID,ZONA_ID,DATA_REFERENCIA,DATA_FUTURA,TEMPERATURA_MAX,TEMPERATURA_MIN,UMIDADE_MAX,UMIDADE_MIN,VENTANIA,PRECIPITACAO,NUVEM,PRESSAO


##### Chamada API

In [83]:
# Parâmetros da API disponíveis na versão gratuita
parameters = {
    't_max_2m_24h:C': 'Temperatura_Max',
    't_min_2m_24h:C': 'Temperatura_Min',
    'relative_humidity_2m:p': 'Umidade_Max',
    'wind_gusts_10m_1h:ms': 'Ventania',
    'precip_24h:mm': 'Precipitacao',
    'total_cloud_cover:octas': 'Nuvem',
    'msl_pressure:hPa': 'Pressao'
}

model = 'mix'
startdate = dt.datetime.utcnow().replace(minute=0, second=0, microsecond=0)
enddate = startdate + dt.timedelta(days=7)
interval = dt.timedelta(hours=1)

In [84]:
# Dicionário para armazenar os dados de cada zona
dados_zonas = {}

In [85]:
for idx, row in Select_TBL_Zona.iterrows():
    zona_geom = loads(row['geometry']) if isinstance(row['geometry'], str) else row['geometry']
    zona_geom_simplificada = zona_geom.simplify(0.01, preserve_topology=True)
    
    coords = [(lat, lon) for lon, lat in zona_geom_simplificada.exterior.coords]
    coords_str = ','.join([f"{lat},{lon}" for lat, lon in coords])
    request_url = f'https://api.meteomatics.com/{startdate.strftime("%Y-%m-%dT%H:%M:%SZ")}/{"".join(parameters.keys())}/polygon({coords_str})/json?model={model}'
    
    print(f"URL de requisição para a zona {row['NOME_ZONA']}: {request_url}")
    
    try:
        df = api.query_time_series(coords, startdate, enddate, interval, list(parameters.keys()), username, password, model=model)
        df.columns = [parameters[col] if col in parameters else col for col in df.columns]
        dados_zonas[row['NOME_ZONA']] = df
    except Exception as e:
        print(f"Erro ao processar a zona {row['NOME_ZONA']}: {e}")

URL de requisição para a zona Centro Ampliado: https://api.meteomatics.com/2024-09-03T20:00:00Z/t_max_2m_24h:Ct_min_2m_24h:Crelative_humidity_2m:pwind_gusts_10m_1h:msprecip_24h:mmtotal_cloud_cover:octasmsl_pressure:hPa/polygon(-23.642215999,-46.674213,-23.6114339989999,-46.718221,-23.617119999,-46.74974,-23.594919999,-46.760663,-23.6090669989999,-46.807775,-23.5840099989999,-46.806459,-23.554834999,-46.760125,-23.520376999,-46.747045,-23.4941559989999,-46.758695,-23.5146879989999,-46.7264089999999,-23.5193029549999,-46.6252015009999,-23.5532970089999,-46.625203848,-23.5545829989999,-46.610221,-23.604665999,-46.579296,-23.645173999,-46.5845909999999,-23.675136999,-46.63358,-23.642215999,-46.674213)/json?model=mix
URL de requisição para a zona Zona Leste 1: https://api.meteomatics.com/2024-09-03T20:00:00Z/t_max_2m_24h:Ct_min_2m_24h:Crelative_humidity_2m:pwind_gusts_10m_1h:msprecip_24h:mmtotal_cloud_cover:octasmsl_pressure:hPa/polygon(-23.6008049989999,-46.575363,-23.5532970089999,-46.625

In [86]:
# Salvar os resultados em um arquivo CSV, por exemplo
for zona, df in dados_zonas.items():
    df.to_csv(f'dados_meteorologicos_{zona}.csv', index=False)

In [89]:
import logging

# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def adjust_value(value, column_name):
    if value is not None:
        value = round(float(value), 2)
    return value

logging.info("Iniciando a inserção de dados.")

for zona_nome, df in dados_zonas.items():
    zona_id = int(Select_TBL_Zona.loc[Select_TBL_Zona['NOME_ZONA'] == zona_nome, 'ID'].values[0])
    df['Zona_Id'] = zona_id
    
    df['Data_Futura'] = pd.date_range(start=startdate, periods=len(df), freq=interval)
    
    for i, row in enumerate(df.iterrows(), start=1):
        sql_insert = """
        INSERT INTO TBL_Previsao_Futura 
        (Zona_Id, Data_Referencia, Data_Futura, Temperatura_Max, Temperatura_Min, Umidade_Max, Ventania, Precipitacao, Nuvem, Pressao)
        VALUES 
        (:Zona_Id, :Data_Referencia, :Data_Futura, :Temperatura_Max, :Temperatura_Min, :Umidade_Max, :Ventania, :Precipitacao, :Nuvem, :Pressao)
        """
        data_insert = {
            'Zona_Id': int(zona_id),
            'Data_Referencia': startdate,
            'Data_Futura': row[1]['Data_Futura'],
            'Temperatura_Max': adjust_value(row[1].get('Temperatura_Max', None), 'Temperatura_Max'),
            'Temperatura_Min': adjust_value(row[1].get('Temperatura_Min', None), 'Temperatura_Min'),
            'Umidade_Max': adjust_value(row[1].get('Umidade_Max', None), 'Umidade_Max'),
            'Ventania': adjust_value(row[1].get('Ventania', None), 'Ventania'),
            'Precipitacao': adjust_value(row[1].get('Precipitacao', None), 'Precipitacao'),
            'Nuvem': row[1].get('Nuvem', None),
            'Pressao': adjust_value(row[1].get('Pressao', None), 'Pressao')
        }
        
        cursor.execute(sql_insert, data_insert)
        logging.info(f"Linha {i} inserida para a zona {zona_nome}.")
    
connection.commit()
finaliza_conexao(cursor, connection)
logging.info("Inserção de dados concluída.")


2024-09-03 17:25:51,310 - INFO - Iniciando a inserção de dados.
2024-09-03 17:25:51,330 - INFO - Linha 1 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,348 - INFO - Linha 2 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,365 - INFO - Linha 3 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,384 - INFO - Linha 4 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,401 - INFO - Linha 5 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,420 - INFO - Linha 6 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,436 - INFO - Linha 7 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,455 - INFO - Linha 8 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,473 - INFO - Linha 9 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,490 - INFO - Linha 10 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,507 - INFO - Linha 11 inserida para a zona Centro Ampliado.
2024-09-03 17:25:51,527 - INFO - Linha 12 inserida para a zona Ce